# Data Wrangling using Dplyr

#####  Tutorial at http://genomicsclass.github.io/book/pages/dplyr_tutorial.html

### Verbs
* Select : To select columns
* Filter : to Filter rows
* Arrange: to order rows
* mutate: to create new columns
* summary : to create summaries 
* group_by: create summaries by group  (Split -> apply -> combine)


In [4]:
library(dplyr)
library(downloader)

In [50]:
url <- "https://raw.githubusercontent.com/genomicsclass/dagdata/master/inst/extdata/msleep_ggplot2.csv"
filename <- "msleep_ggplot2.csv"

if (!file.exists(filename)) download(url,filename)
    
msleep <- read.csv("msleep_ggplot2.csv")
head(msleep)

name,genus,vore,order,conservation,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
Cheetah,Acinonyx,carni,Carnivora,lc,12.1,NA,NA,11.9,NA,50.000
Owl monkey,Aotus,omni,Primates,NA,17.0,1.8,NA,7.0,0.01550,0.480
Mountain beaver,Aplodontia,herbi,Rodentia,nt,14.4,2.4,NA,9.6,NA,1.350
Greater short-tailed shrew,Blarina,omni,Soricomorpha,lc,14.9,2.3,0.1333333,9.1,0.00029,0.019
Cow,Bos,herbi,Artiodactyla,domesticated,4.0,0.7,0.6666667,20.0,0.42300,600.000
Three-toed sloth,Bradypus,herbi,Pilosa,NA,14.4,2.2,0.7666667,9.6,NA,3.850


# Selecting columns using select()

In [ ]:
# Select some columns
sleepData <- select(msleep,name,sleep_total)
head(sleepData)

In [ ]:
# Select all but name
head(select(msleep,-name))

In [ ]:
# Select all from name to order
head(select(msleep, name: order))

In [ ]:
# select columns that start with sleep
head(select(msleep, starts_with("sleep")))

In [ ]:
# select columns that end with t
head(select(msleep, ends_with("t")))

In [ ]:
# select columns that contain sleep
head(select(msleep, contains("sleep")))

# Select rows using filter()

In [ ]:
# filter on total sleep
head(filter(msleep,sleep_total >= 16))

In [ ]:
# filter on total sleep and bodywt
head(filter(msleep, sleep_total >= 16, bodywt >= 1))

In [ ]:
# same with just and operator
head(filter(msleep, sleep_total >= 16 & bodywt >= 1))

In [ ]:
# filter where order is in the list
head(filter(msleep, order %in% c('Cingulata','Didelphimorphia')))

# Pipe operator %>%

In [ ]:
msleep %>%
filter(order %in% c("Cingulata"), sleep_total > 10)%>%
select(name,genus,order)%>%
head()


# Reorder rows using arrange()

In [ ]:
# order by column order ascending is default
msleep %>% arrange(order) %>% head()

In [ ]:
# select some columns and then order 
msleep%>%
    select(name,order,sleep_total)%>%
    arrange(order,desc(sleep_total))%>%
    head()

In [ ]:
# select columns, arrange them and then filter
msleep%>%
    select(name,order,sleep_total)%>%
    arrange(order,desc(sleep_total))%>%
    filter(sleep_total>15)%>%
    head()

# Add new column using mutate

In [ ]:
# create a new column using two exisiting columns
# then filter and arrange and then select
msleep %>%
    mutate(remproportion = sleep_rem/sleep_total, bodywt_grams = bodywt * 1000)%>%
    filter(remproportion >= 0.2)%>%
    arrange(desc(remproportion))%>%
    select(name:order,remproportion,bodywt_grams)%>%
    head

# Create summaries of data using summarise()

In [43]:
# create summary over the whole data
msleep %>%
    summarise(
        avg_sleep = mean(sleep_total),
        min_sleep = min(sleep_total),
        max_sleep = max(sleep_total),
        total =n())

avg_sleep,min_sleep,max_sleep,total
10.43373,1.9,19.9,83


In [44]:
# filter and then summarise
msleep %>%
    filter(genus %in% c('Canis','Tupaia'))%>%
    summarise(avg_sleep = mean(sleep_total))%>%
    head

avg_sleep
9.5


# Group operations using group_by()

In [49]:
# split on column -> apply functions -> combine results

# group by order and then summarise
msleep %>%
    group_by(order)%>%
    summarise(avg_sleep = mean(sleep_total),
            min_sleep = min(sleep_total),
             max_sleep= min(sleep_total),
             records = n())%>%
    head

order,avg_sleep,min_sleep,max_sleep,records
Afrosoricida,15.600000,15.6,15.6,1
Artiodactyla,4.516667,1.9,1.9,6
Carnivora,10.116667,3.5,3.5,12
Cetacea,4.500000,2.7,2.7,3
Chiroptera,19.800000,19.7,19.7,2
Cingulata,17.750000,17.4,17.4,2
